# Úvod
Regulární výrazy jsou seskupení znaků a znaky zastupujících útvarů, které lze použít k nalezení určitých řetězců v textu. Člověk se s nimi může setkat ve všelijakých aplikacích - existují na nich zcela stojící programy (např. SED), jsou součástí textových editorů (např. Notepad++) a odpovídající knihovny najdeme i v programovacích jazycích (např. balíček *re* v Pythonu).  
Regulární výrazy jsou velice mocný nástroj. Neměly by se ale používat všude. Právě naopak, šáhnout bychom po nich měli až když selžou všechny ostatní metody. Například pokud bychom chtěli parsovat XML soubor, měli by se použít balíčky/knihovny/aplikace využívající XPath. U jiných problémů zase vyvstává otázka, zda není rozumější nadefinovat si vlastní gramatiku např. pomocí balíčku *pyparsing*. Regulární výrazy totiž jednak nebývají výpočetně zrovna nejefektivnější, jednak tendují k značné nepřehlednosti.  
Různé aplikace používající regulární výrazy je podporují do různé míry a i samotný způsob, jak s nimi uživatel operuje, bývá odlišný. Proto se v tomto workshopu budeme věnovat regulárním výrazům nikoli pouze v Pythonu, ale i v Notepadu++ a v Teradatě.  

## Obsah
- [Python](#Python)  
    - [Funkce search a objekt Match](#Funkce-search-a-objekt-Match)
    - [Speciální znaky v regulárních výrazech](#Speciální-znaky-v-regulárních-výrazech)
        - [Tečka a funkce findall a finditer](#Tečka-a-funkce-findall-a-finditer)
        - [Bílé znaky](#Bílé-znaky)
        - [Písmena a číslice](#Písmena-a-číslice)
    - [Hranaté závorky alias explicitní výběr](#Hranaté-závorky-alias-explicitní-výběr)
    - [Specifikování počtu znaků](#Specifikování-počtu-znaků)
    - [OR pattern](#OR-pattern)
    - [Začátek a konec stringu/řádky](#Začátek-a-konec-stringu/řádky)
    - [Začátek a konec slova](#Začátek-a-konec-slova)
    - [Grupy](#Grupy)
    - [Grupy a flagy](#Grupy-a-flagy)
    - [Lookahead a comp.](#Lookahead-a-comp.)
    - [Další Pythoní regexové funkce](#Další-Pythoní-regexové-funkce)
        - [Match a fullmatch](#Match-a-fullmatch)
        - [Split](#Split)
        - [Sub a subn](#Sub-a-subn)
        - [Compile](#Compile)
    - [Příklady použití regexů](#Příklady-použití-regexů)
        - [Vypreparování HTML tagů](#Vypreparování-HTML-tagů)
        - [Prozkoumání URL](#Prozkoumání-URL)
        - [Nalezení čísel v textu](#Nalezení-čísel-v-textu)
        - [Parsování csv](#Parsování-csv)
        - [Práce se speciálními znaky](#Práce-se-speciálními-znaky)
        - [Hledání sloupců bez aliasu v SQL](#Hledání-sloupců-bez-aliasu-v-SQL)
    - [Fuzzy search](#Fuzzy-search)
- [Notepad++](#Notepad++) 
- [Teradata](#Teradata) 
    - [REGEXP_SUBSTR](#REGEXP_SUBSTR)
    - [REGEX_REPLACE](#REGEX_REPLACE)
    - [REGEX_INSTR](#REGEX_INSTR) 
    - [REGEX_SIMILAR](#REGEX_SIMILAR) 
    - [REGEX_SPLIT_TO_TABLE](#REGEX_SPLIT_TO_TABLE) 

## Python

Knihovna zajišťující podporu pro používání regulárních výrazů v Pythonu se nazývá *re*. Je součástí základní instalace, tudíž ji nemusíme stahovat pipem.

In [1]:
import re

### Funkce search a objekt Match
Než začneme probírat jednotlivá zákoutí regulárních výrazů, musíme se seznámit s fungováním funkce *search*. Ta v případě, kdy je hledaný předpis - pattern - v prohledávaném textu nalezen, vrátí objekt typu *Match*. V případě neúspěchu je ale výsledkem *None*. Běžně se tak úspěch nalezení patternu kontroluje s pomocí *if* podmínky. Jelikož totiž Match objekt nemá zadefinovanou ani funkci *\_\_bool__*, ani funkci *\_\_len__*, má z hlediska if podmínky hodnotu True (viz [zde](https://docs.python.org/3/library/stdtypes.html#truth-value-testing)).

In [4]:
original_text = (
    "If the swift moment I entreat:\n"
    "Tarry a while! You are so fair!\n"
    "Then forge the shackles to my feet,\n"
    "Then I will gladly perish there!\n"
    "Then let them toll the passing-bell,\n"
    "Then of your servitude be free, "
    "The clock may stop, its hands fall still,\n"
    "And time be over then for me!"
    )
word_servitude_matched = re.search("servitude", original_text)
word_eternity_matched = re.search("eternity", original_text)
if word_servitude_matched:
    print("Word 'servitude' is in the text.")
if word_eternity_matched:
    print("Word 'eternity' is in the text.")
else:
    print("Word 'eternity' is not in the text.")

Word 'servitude' is in the text.
Word 'eternity' is not in the text.


Co všechno vlastně můžeme s Match objektem dělat? V prvé řadě je v něm uložena kopie textového řetězce, který byl prohledáván. Dostaneme se k němu s pomocí atributu *string*

In [6]:
word_servitude_matched.string

'If the swift moment I entreat: Tarry a while! You are so fair! Then forge the shackles to my feet, Then I will gladly perish there! Then let them toll the passing-bell, Then of your servitude be free, The clock may stop, its hands fall still, And time be over then for me!'

Pokud bychom naopak potřebovali znát pattern, který jsme v textu hledali, použijeme atribut *re*.

In [7]:
word_servitude_matched.re

re.compile(r'servitude', re.UNICODE)

Pokud chceme vědět, na kterých pozicích v textu nalezený blíženec patternu začínal a končil, aplikujeme funkci *span*.

In [10]:
word_servitude_matched.span()

(182, 191)

Chceme-li vidět nalezené slovo, použijeme funkci *group*. V tomto konkrétním příkladě je přidaná informace pochopitelně nulová, užitečné to bude ale v případě obecnějších patternů. K tomu, co to vlastně ony grupy jsou, se dostaneme v textu dále.

In [13]:
word_servitude_matched.group()

'servitude'

Funkci *search* stejně jako podobným funkcím popsaným v textu dále lze jako parametry předat kromě patternu a prohledávaného textu i *flag*. Jedná se o jednu či více hodnot nějak funkčnost regexovského enginu modifikující. Kupříkladu defaultně jsou regulární výrazy *case sensitive*, tj. velká a malá písmena jsou braná jako rozdílné entity. S použitím *re.I* (resp. *re.IGNORECASE* - téměř každý flag má krátký a dlouhý název) to už ale neplatí.  
Nejprve se podívejme na situaci bez *re.I*:

In [5]:
flag_i_not_present = re.search("ABCD", "abcd")
if flag_i_not_present:
    print(flag_i_not_present.group())
else:
    print("Pattern not present")

Pattern not present


Nyní flag přítomen je:

In [6]:
flag_i_present = re.search("ABCD", "abcd", re.I)
if flag_i_present:
    print(flag_i_present.group())
else:
    print("Pattern not present")

abcd


Z dalších flagů krátce zmiňme 
- re.S (re.DOTALL) - tečka zastupuje i "znak" nového řádku (defaultně tečka zastupuje všechno kromě onoho znaku).
- re.M (re.MULTILINE) - stříška a dolar zastupují nejen začátek a konec celého textu, ale i začátek a konec řádky.

Příklady jejich použítí ukážeme při diskutování s nimi spojených znaků.

Pokud chceme použít naráz více flagů, musíme mezi ně vložit pipu "|" (pravý alt + W):

In [14]:
flags_matched = re.search("ABCD", "abcd", re.I|re.S)
if flags_matched:
    print(flags_matched.group())
else:
    print("Pattern not present")

abcd


### Speciální znaky v regulárních výrazech

#### Tečka a funkce findall a finditer
Pakliže bychom potřebovali nalézt v textu jedno konkrétně specifikované slovo, asi bychom o regulárních výrazech vůbec neuvažovali. Představme si proto situaci, kdy chceme nalézt všechna slova tvořená pěti písmeny, za kterými bezprostředně následuje vykřičník. V takovém případě použijeme třeba tečku (přesněji tedy pět teček následovaným vykričníkem) - ta zastoupí jakýkoli znak kromě nového řádku. Pokud přidáme flag *re.S*, zastoupí i nový řádek. 

In [7]:
five_letter_word_matched = re.search(".....!", original_text)
if five_letter_word_matched:
    print(five_letter_word_matched.group())
else:
    print("Nothing has been found!")

while!


Vidíme, že vskutku bylo nalezelo pětipísmené slovo následované vykřičníkem. Jenže... v textu jsou přece taková slova minimálně dvě - kromě "while" je tam i "there". *Search* totiž najde pouze první výskyt patternu. Pokud chceme všechny výskyty, musíme použít *findall*. Ten nevrací Match objekt, ale list stringů. Pokud pattern není v textu nalezen, vrátí se list prázdný.

In [8]:
five_letter_word_matched_2 = re.findall(".....!", original_text)
print(five_letter_word_matched_2)

['while!', ' fair!', 'there!', 'or me!']


Vidíme, že jsme se trochu přepočítali - tečka totiž zastupuje jakýkoli znak - včetně **mezery**. K tomuto problému se vrátíme za chvíli, nyní se ještě věnujme funkcím. Co kdybychom totiž potřebovali všechny výskyty patternu a současně bychom nechtěli stringy, ale Match objekty? V takovém případě použijeme funkci *finditer*. Ta vrátí iterátor jdoucí přes všechny (nepřekrývající se) blížence patternu v textu.

In [9]:
five_letter_word_matched_3 = re.finditer(".....!", original_text)
for match_object in five_letter_word_matched_3:
    print(match_object.group())

while!
 fair!
there!
or me!


Mohlo by nás napadnout, co dělat, když chceme mít v patternu "normální" znak tečky. V takovém případě ji musíme stejně jako další podobné speciální znaky odescapovat pomocí zpětného lomítka. Jelikož je ale zpětné lomítko speciální znak pro samotný Python, musí se zde psát zpětné lomítka dvě. Python na ně narazí a zprocesuje je - výsledkem je jedno zpětné lomítko. To pak využijí regulární výrazy. V tomhle konkrétním příkladě by pravda stačilo použít jen jedno lomítku, jelikož v Pythonu samotném nemá "\\." narozdíl například od "\n" žádný speciální význam. Nicméně je lepší na takovouto náhodu nespoléhat.

In [9]:
five_letter_word_dot_matched = re.finditer(".....\\.", "Something. Hm. Abcde. Hm....")
for match_object in five_letter_word_dot_matched:
    print(match_object.group())

thing.
Abcde.
 Hm...


#### Bílé znaky
No, je vidět, že se musíme naučit, kterak do patternu zanést bílé znaky. Tímto označením myslíme mezery, tabulátory, znaky nových řádků atd. Ty jsou všechny v regexech zastoupeny prostřednictvím značky "\s". Zkusme s její pomocí opravit hledání pětipísmenového slova následovaného vykřičníkem.

In [10]:
really_five_letter_word_matched = re.finditer("\\s.....!", original_text)
for match_object in really_five_letter_word_matched:
    print(match_object.group())

 while!
 there!


Sláva, funguje to!  
Zmiňme nakonec, že existuje i komplementární "\S". To zastoupí cokoli, co **není** bílý znak.

#### Písmena a číslice
Pakliže existuje speciální "znak" pro znaky bíle, není k mání něco podobného pro písmena a číslice? Inu, kdyby nebylo, tak takto odstavec nezačínáme. Cifry (tj. 0 až 9) zastupuje "\d". Striktně pro písmena nic neexistuje. Máme sice k dispozici "\w", do něho ale kromě písmen (velkých, malých a defaultně i českých) patří i cifry a podtržítko. To vidíme na následujícím příkladu:

In [11]:
two_w_matched = re.finditer("\\s\\w\\w", "something ab 12 anything 3E čš _ý __")
for match_object in two_w_matched:
    print(match_object.group())

 ab
 12
 an
 3E
 čš
 _ý
 __


Podobně jako u "\s" existují i pro "\d" a "\w" komplementární "\D" a "\W".

### Hranaté závorky alias explicitní výběr
Nicméně fakt, že písmena nemají svého vlastního zástupce, by nás neměl zastavit. Lze si totiž manuálně vytvořit množinu potenciálně použitelných znaků. Ty se napíšou bez jakéhokoli oddělování do hranatých závorek. Příklad:

In [12]:
sad_mad_matched = re.finditer("[sm]ad", "Am I sad or am I mad?")
for match_object in sad_mad_matched:
    print(match_object.group())

sad
mad


Tudíž pakliže bychom chtěli mít zástupce jen a pouze pro písmena, uvedli bychom je všechna explicitně do hranatých závorek. No... takovýto přístup by fungoval, ale moc elegantní by to nebylo. Naštěstí do hranatých závorek se dají psát s pomocí znaku "-" i rozsahy podle ASCII/Unicode tabulky. Všechny písmena bez diakritiky tak zastupuje [a-zA-Z], s diakritikou pak:

In [2]:
two_letters_matched = re.finditer("[a-zA-Zá-žÁ-Ž][a-zA-Zá-žÁ-Ž]", "ab 3c De šč")
for match_object in two_letters_matched:
    print(match_object.group())

ab
De
šč


Mohlo by Vás napadnout, co dělat, když chceme mít v množině znaků mínus anebo zavírající hranatou závorku. Jde to, jen se musí tyhle speciální znaky zpětným lomítkem odescapovat:

In [3]:
two_letters_matched_2 = re.finditer("[a-zA-Z\\]\\-][a-zA-Z\\]\\-]", "ab p- o]")
for match_object in two_letters_matched_2:
    print(match_object.group())

ab
p-
o]


V hranatých závorkách se mohou objevit i znaky, které mají v regulárích výrazech specifický význam, například hvězdička, tečka či otazník. V takovém případě tu jsou tyto znaky ale samy za sebe a speciální význam nemají. Escapování tak není potřeba.

In [15]:
special_chars_matched = re.finditer("[a-zA-Z.?][a-zA-Z.?]", "ab p? o5 .u")
for match_object in special_chars_matched:
    print(match_object.group())

ab
p?
.u


V množině znaků se můžou objevit i věci typu "\d" či "\w"

In [16]:
special_chars_matched_2 = re.finditer("[\\d.][\\d.]", "ab 45 o5 .1")
for match_object in special_chars_matched_2:
    print(match_object.group())

45
.1


Co ale máme udělat, když chceme specifikovat množinu znaků, které **NE**chceme? V takovém případě musíme bezprostředně za otevírací hranatou závorku napsat stříšku, tj. ^. Pakliže bychom ji napsali až jako druhou, ztratila by svůj speciální význam a brala by se jako normální znak.  
V příkladě hledáme mezeru následovanou libovolným znakem kromě 1, 3 a 5; poté následuje libovolné číslo.

In [19]:
forbidden_chars_matched = re.finditer("\\s[^135][\\d]", " 12 45 65 31")
for match_object in forbidden_chars_matched:
    print(match_object.group())

 45
 65


### Specifikování počtu znaků

Při demonstranci možností tečky coby speciálního regexovského znaku jsme pro pětipísmené slovo těch teček napsali pět. Takto ručně zadávat znak po znaku by bylo únavné a hlavně by podobný postup vedl ke zbytečným chybám. Naštěstí počet znaků můžeme specifikovat pomocí složených závorek, do nichž se ono číslo zapíše. Samotné složené závorky pak těsně následují za znakem, ke kterému se vážou.  
V příkladu chceme nalézt všechny řetězce, které se skládají z pětí písmen/číslic/podtržítek a za nimiž následuje vykřičník.

In [20]:
five_letter_word_matched_again = re.finditer("\\w{5}!", original_text)
for match_object in five_letter_word_matched_again:
    print(match_object.group())

while!
there!


Pokud chceme specifikovat nikoli přesný počet znaků, nýbrž interval jejich počtu, opět použijeme složené závorky. Do nich uvedeme počátek a konec intervalu oddělené čárkou.  
V příkladu hledáme řetězce, kde se před vykřičníkem nalézá 4 až 6 (oboje včetně) písmen/číslic/podtržítek.

In [21]:
four_to_six_letter_word_matched = re.finditer("\\w{4,6}!", original_text)
for match_object in four_to_six_letter_word_matched:
    print(match_object.group())

while!
fair!
there!


Pakliže bychom chtěli regexovému enginu říci, že určitý znak se vyskytuje jednou nebo vůbec, mohli bychom psát {0,1}. Nicméně tento velice častý případ má svůj vlastní speciální znak - otazník. Ten je "greedy", tj. pokud to jen trochu jde, pokusí se onen jeden znak matchnout.    
V příkladu se snažíme zjistit, zda se v textu vyskytuje slovo "hand", jedno jestli v jednotném či v množném čísle.

In [22]:
hand_hands_matched = re.finditer("hands?", original_text)
for match_object in hand_hands_matched:
    print(match_object.group())

hands


Zde se otazník vztahoval pouze k písmenu s. Nicméně vypustit ho můžeme i na skupinu znaků - pokud je předtím umístíme do kulatých závorek. Například pokud bychom chtěli matchnout jak "pass", tak "passing", použijeme

In [23]:
pass_passing_matched = re.finditer("pass(ing)?", original_text)
for match_object in pass_passing_matched:
    print(match_object.group())

passing


Velice často budeme chtít v patternech použít určitý znak či skupinu znaků libovolněkrát a to klidně i nulakrát. V takovém případě za znak/skupinu znaků napíšeme hvězdičku.  
V příkladu níže hledáme všechna slova (s libovolným počtem písmen), za kterými následuje čárka.

In [24]:
before_comma_matched = re.finditer("\\w*,", original_text)
for match_object in before_comma_matched:
    print(match_object.group())

feet,
bell,
free,
stop,
still,


Nakonec existuje i speciální znak *+*, který říká, že jemu předcházející znak se může objevit libovolněkrát, avšak aspoň jednou.

Značky + a * jsou stejně jako otazník "greedy", tj. snaží se namatchovat co možná nejdelší část textu. Někdy ale takové chování není chtěné. Představme si například, že bychom chtěli regulárními výrazy získat obsah nějakého tagu a měli k dispozici text "&lt;a&gt; b &lt;c&gt;".  
Při použití greedy klasifikátorů si můžeme představovat, že se klasifikátor chytne na první znak &lt; a posléze při matchování .* do onoho klasifikátoru vloží celý zbytek stringu včetně &gt;. Pak kontroluje poslední znak patternu, kde má znak &gt;. Regex engine koukne na string a zjišťuje, že mu už žádný znak k matchnutí nezbývá. Donutí tedy hvězdičku jeden znak vrátit. Jelikož to bude právě &gt; poslední znak patternu se úspěšně matchne.

In [26]:
too_greedy_match = re.finditer("<.*>", "<a> b <c>")
for match_object in too_greedy_match:
    print(match_object.group())

<a> b <c>


Jenže my chtěli získat opravdu jen ony tagy, tj. &lt;a&gt; a &lt;c&gt;. K tomu použijeme líný kvantifikátor, který si nevezme maximum, ale minimum možného. Líné kvantifikátory hvězdy a plusu se jinak chovají jako jejich hltavé protějšky. Do patternu se zapíší jako *? a +?.  
K příkladu je ještě vhodné zdůraznit, že pokud bychom vynechali v patternu uzavírací zobáček, líná hvězdička by nematchla nic a výsledkem by byly by matche jen a pouze znaku &lt;.

In [27]:
too_greedy_match = re.finditer("<.*?>", "<a> b <c>")
for match_object in too_greedy_match:
    print(match_object.group())

<a>
<c>


### OR pattern
Je možné, že budeme potřebovat namatchovat text buďto jednoho, nebo druhého tvaru. V takovém případě oba potenciální patternny oddělíme pipou, tj. tímto: "|" (pravý alt + W). Na příkladu níže vidíme, že i bez použití závorek se bere vše, co je po stranách pipy, a nikoli jen jeden znak nalevo a jeden znak napravo od ní. 

In [28]:
or_match = re.finditer("\\s\\w{4}!|\\s\\w{4},", original_text)
for match_object in or_match:
    print(match_object.group())

 fair!
 feet,
 free,
 stop,


V příkladu byly patterny v ORu vzájemně neslučitelné - každé slovo je bezprostředně následováno čárkou anebo vykřičníkem, nikoli obojím. Co ale nastane, když by mohly platit patterny oba? Zkusme regexovému enginu říci, že hledáme buďto slovo "shackle", anebo slovo "shackles". V textu je to druhé. Vidíme ale, že nalezen byl blíženec patternu prvního. To proto, že pokud je první složka ORu úspěšně matchnuta, druhá už není kontrolována a to ani když by vedla k delšími matchi. Dá se tedy říci, že kvantifikátor OR je vždy líný.

In [29]:
or_match = re.finditer("shackle|shackles", original_text)
for match_object in or_match:
    print(match_object.group())

shackle


### Začátek a konec stringu/řádky
Když chceme v patternu říci, že něco následuje bezprostředně po začátku textu, použijeme značku pro tento začátek - stříšku. Obdobně konec textu se označí dolarem.

In [22]:
start_string_match = re.finditer("^\\w*", original_text)
for match_object in start_string_match:
    print(match_object.group())

If


Obvykle ale budeme chtít mít v patternu zanesený nikoli začátek celého textu, ale začátek řádku. V takovém případě musíme použít flag MULTILINE.

In [35]:
start_string_match = re.finditer("^\\w*", original_text, re.MULTILINE)
for match_object in start_string_match:
    print(match_object.group())

If
Tarry
Then
Then
Then
Then
And


### Začátek a konec slova
Podobně jako pro začátek stringu/řádky máme i speciální znak pro začátek a konec slova. Zde se v obou případech používá "\b". De facto tak zajišťuje podporu pro vyhledávání celých slov. Formálně je "\b" zadefinované jako hranice mezi znakem typu "\w" a znakem typu "\W", resp. začátkem/koncem prohledávaného textu.  
V příkladu níže lze nalézt řetězec "konec" jak ve slově "konec", tak ve slově "koneckonců", a jsou tak nalezeny dvě shody.

In [36]:
without_word_boundary_match = re.finditer("konec", "koneckonců konec ještě neskončil")
for match_object in without_word_boundary_match:
    print(match_object.group())

konec
konec


S použitím "\b" už je nález jediný.

In [37]:
with_word_boundary_match = re.finditer("\\bkonec\\b", "koneckonců konec ještě neskončil")
for match_object in with_word_boundary_match:
    print(match_object.group())

konec


### Grupy
Na začátku kapitoly jsme hledali slova, po kterých následuje vykřičník. Slova se nám sice na výstupu objevila, ale bohužel i s vykřičníkem. Co dělat, když ten ve výsledku hledání nechceme? Lze použít grupy. Jedná se de facto o uzávorkování určité části patternu.

In [38]:
group_zero_match = re.finditer("(\\b\\w+\\b)!", original_text)
for match_object in group_zero_match:
    print(match_object.group())

while!
fair!
there!
me!


Závorkování jsme použili, ale výsledek je stejný! Kde se stala chyba? Defaultně se nám vytiskla grupa 0, ve které je zahrnut celý pattern. Když chceme pouze obsah n-té závorky, musíme číslo n napsat do funkce *group*. 

In [34]:
group_one_match = re.finditer("(\\b\\w+\\b)!", original_text)
for match_object in group_one_match:
    print(match_object.group(1))

while
fair
there
me


Na čísla grup se lze odkazovat i v samotném patternu. To je užitečné nejen pro změnšení počtu úderů do klávesnice. Představme si, že v rámci odhalování překlepů chceme najít všechny dvojice těsně za sebou následujících stejných slov. Pro jedno slovo by se dalo použít následující:  

In [39]:
duplicated_what_match = re.finditer("\\bwhat\\b\\s+\\bwhat\\b", "something anything what what so we we")
for match_object in duplicated_what_match:
    print(match_object.group())

what what


Nicméně takovouto operaci nemůžeme provádět pro každé slovo ze slovníku. To, co můžeme provést, je odkaz na obsah grupy a to pomocí zpětného lomítka a čísla grupy. Zdůrazněme, že odkaz se odkazuje na skutečný obsah grupy v době běhu regexového enginu a ne pouze na obecný předpis v patternu.

In [41]:
duplicated_word_match = re.finditer("(\\b\\w+\\b)\\s+\\1", "something anything what what so we we")
for match_object in duplicated_word_match:
    print(match_object.group(1))

what
we


Grupy se hodí i k organizaci a zpřehlednění patternů například u ORu. Asi ještě důležitejší však je, že celá grupa může být najednou ovlivněna znaky typu otazníku, plusu či hvězdy. V takovém případě se ale může stát, že nás obsah každé jednotlivé z nich nezajímá a jen bychom museli v hlavě držet zbytečně mnoho čísel a vědět, která jsou relevantní a která ne.  
V příkladu bychom chtěli matchnout první a druhé slovo od začátku prohledávaného textu. Jenže bohužel regexový engine bere jako první grupu tu obě slova obsahující. 

In [42]:
too_many_groups_match = re.finditer("^((\\b\\w+\\b)\\s+(\\b\\w+\\b))", original_text)
for match_object in too_many_groups_match:
    print("First group: ", match_object.group(1), "; Second group: ", match_object.group(2))

First group:  If the ; Second group:  If


Naštěstí se enginu dá říci, že danou grupu chceme z číslování vyřadit. To se provede tak, že těsně za její otevírací závorku napíšeme otazník a dvojtečku:

In [43]:
too_many_groups_match = re.finditer("^(?:(\\b\\w+\\b)\\s+(\\b\\w+\\b))", original_text)
for match_object in too_many_groups_match:
    print("First group: ", match_object.group(1), "; Second group: ", match_object.group(2))

First group:  If ; Second group:  the


Pokud by nám nevyhovovala identifikace grup pomocí čísel, lze jim přiřadit i názvy. To se provede tak, že se na začátek grupy umístí otazník, velké P a pak se mezi &lt; a &gt; vloží samotné její jméno. Pakliže se chceme ještě v patternu na obsah grupy odkazovat, tak krom zpětného lomítka a čísla můžeme do kulatých závorek napsat ?P= a název grupy.

In [44]:
duplicated_word_match = re.finditer("(?P<duplikace>\\b\\w+\\b)\\s+(?P=duplikace)", "something anything what what so we we")
for match_object in duplicated_word_match:
    print(match_object.group("duplikace"))

what
we


### Grupy a flagy

Občas člověk narazí na to, že je v kulatých závorkách otazník následovaný nějakým písmenem, nejčastěji asi malým i. Tímto způsobem lze zapínat stejnou funkcionalitu, kterou obstarávají flagy. Tj. *i* je spojené s (ne)citlivostí na velikost písmen. Podobně by *m* mělo fungovat jako re.M či *s* jako re.S.  
Nejprve si vezměme případ, kdy žádný přepínač ani flag není přítomný. Pattern se nám zde na text nenamatchuje.

In [46]:
group_flag_matched_1 = re.finditer("ABCD ABCD", "abcd abcd")
for match_object in group_flag_matched_1:
    print(match_object.group())

Nyní na začátek patternu vložíme grupu-přepínač a na velikost písmen se už nehledí. Mimochodem i když je otazník s písmenem i v závorkách, do výčtu číslovaných grup se nedostane. Tj. kdybychom do parametrů funkce group vložili jedničku, skončí kód pádem, stejně jako by v patternu žádná grupa nebyla.

In [48]:
group_flag_matched_2 = re.finditer("(?i)ABCD ABCD", "abcd abcd")
for match_object in group_flag_matched_2:
    print(match_object.group())

abcd abcd


Více stejných grup nepovede k ničemu zajímavého, tj. například nenastane opětovné zapnutí citlivosti na velikost písmen. Maximálně se objeví deprecation hláška o tom, že by se takovéto věci neměly objevovat uprostřed patternu.

In [49]:
group_flag_matched_3 = re.finditer("(?i)ABCD (?i)ABCD", "abcd abcd")
for match_object in group_flag_matched_3:
    print(match_object.group())

abcd abcd


<ipython-input-49-658a5dce7334>:1: DeprecationWarning: Flags not at the start of the expression '(?i)ABCD (?i)ABCD'
  group_flag_matched_3 = re.finditer("(?i)ABCD (?i)ABCD", "abcd abcd")


Co když ale chceme takovéto pseudoflagové chování aplikovat jen na část patternu? V takovém případě onu část napíšeme rovnou do závorek; od flagového písmene bude pattern oddělený dvojtečkou.

In [50]:
group_flag_matched_4 = re.finditer("(?i:ABCD) abcd", "abcd abcd")
for match_object in group_flag_matched_4:
    print(match_object.group())

abcd abcd


Na část patternu mimo závorku nemá zrušení citlivosti na velikost písmen žádný vliv. V příkladu níže tak žádný match nalezen není. Mimochodem ani nyní nedochází k zařazení grupy do číslovaných grup.

In [51]:
group_flag_matched_5 = re.finditer("(?i:ABCD) ABCD", "abcd abcd")
for match_object in group_flag_matched_5:
    print(match_object.group())

### Lookahead a comp.

Opět se jedná o nečíslované grupy. Stejně jako klasický pattern všechny provádějí kontrolu nějakého pravidla. To když neplatí, tak match nenastane. Když ale pravidlo platí, kontrolovaný řetězec patternem zkonzumován není.  
*Lookahead* kontroluje, zda část patternu v závorkách následující za otazníkem a rovnítkem následuje za aktuální pozicí. V příkladu vidíme, že de facto kontrolujeme, zda je v textu přítomno "hands", ale v Match objektu je jenom "hand".

In [52]:
lookahead_matched = re.finditer("\\bhand(?=s\\b)", original_text)
for match_object in lookahead_matched:
    print(match_object.group())

hand


Kdyby v textu bylo pouze "hand", k matchi nedojde:

In [53]:
lookahead_not_matched = re.finditer("\\bhand(?=s\\b)",
                                    ("To bed, to bed! there's knocking at the gate: "
                                     "come, come, come, come, give me your hand. What's "
                                     "done cannot be undone.")
                                   )
for match_object in lookahead_not_matched:
    print(match_object.group())

Pro negativní lookahead match nastane jen tehdy, když pattern v něm uvedený za aktuální pozicí nenásleduje. Lze ho tedy použít na negaci regulárního výrazu. Z hlediska zápisu u něj namísto "?=" na začátek závorek vložíme "?!".

In [54]:
negative_lookahead_not_matched = re.finditer("\\bhand(?!s\\b)", original_text)
for match_object in negative_lookahead_not_matched:
    print(match_object.group())

In [55]:
negative_lookahead_matched = re.finditer("\\bhand(?!s\\b)",
                                        ("To bed, to bed! there's knocking at the gate: "
                                         "come, come, come, come, give me your hand. What's "
                                         "done cannot be undone.")
                                        )
for match_object in negative_lookahead_matched:
    print(match_object.group())

hand


Obdobně fungují lookbehind (zápis "?<=") a negativní lookbehind (zápis "?<!"). Pouze se zaměřují ne na to, co následuje **za** aktuální pozicí, ale na to, co je **před** aktuální pozicí.

### Další Pythoní regexové funkce

Doposud jsme zmínili funkce
- *search* - prohledává celý text, dokud nenajde první výskyt stringu odpovídajícího požadovanému patternu. Vrací *Match* objekt resp. *None*.
- *findall* - vrací všechny nepřekrývající se části textu, které odpovídají patternu, a to jako list stringů. Případně pokud jsou v patternu použity grupy, vrací list tuplů. Nicméně v tuplech je jenom obsah grup - viz příklad níže.
- *finditer* - opět vrací všechny nepřekrývající se části textu, které odpovídají patternu. Tentokrát je ale návratovou hodnotou iterátor s Match objekty.

In [21]:
findall_and_groups_match = re.findall("^(\\b\\w+\\b)\\s+(\\b\\w+\\b)\\s+\\b\\w+\\b", original_text, re.MULTILINE)
print(findall_and_groups_match)

[('If', 'the'), ('Tarry', 'a'), ('Then', 'forge'), ('Then', 'I'), ('Then', 'let'), ('Then', 'of'), ('And', 'time')]


V balíčku *re* ale existuje funkcí více. Podívejme se na ně.  
#### Match a fullmatch
Vrátí výskyt patternu v textu v podobě Match objektu - pattern ale musí začínat už na začátku textu. Tím se liší od *search*e, který pattern objeví v textu kdekoli.  
Nejprve si zkusme z textu matchnout první slovo:

In [7]:
match_match = re.match("If", original_text)
if match_match:
    print(match_match.group())
else:
     print("Pattern not present")   

If


To proběhlo úspěšně - druhé slovo už ale nalezeno nebude:

In [9]:
match_not_match = re.match("the", original_text)
if match_not_match:
    print(match_not_match.group())
else:
     print("Pattern not present")   

Pattern not present


Mimochodem ani v případě multiline flagu by se match neodpoutal od začátku textu.

In [13]:
multiline_match_not_match = re.match("Tarry", original_text, re.MULTILINE)
if multiline_match_not_match:
    print(multiline_match_not_match.group())
else:
     print("Pattern not present")  

Pattern not present


Funkce fullmatch funguje podobně jako match s tím omezením, že se musí matchnout **celý** text.

In [15]:
fullmatch_match = re.fullmatch("If", "If")
if fullmatch_match:
    print(fullmatch_match.group())
else:
     print("Pattern not present")  

If


In [16]:
fullmatch_not_match = re.fullmatch("If", original_text)
if fullmatch_not_match:
    print(fullmatch_not_match.group())
else:
     print("Pattern not present")  

Pattern not present


#### Split
Funkce split rozdělí text podle patternu a vrátí jej v podobě listu stringů.

In [22]:
re.split(",\\s*", "prvni, druhy,     treti")

['prvni', 'druhy', 'treti']

Pakliže se pattern nebo jeho části dá do grupy, bude v výsledném stringu i její obsah. Závorky zde opravdu fungují jako grupa, tj. stačí na začátek napsat "?:" a v listu se jejich obsah opět neobjeví.

In [23]:
re.split("(,)\\s*", "prvni, druhy,     treti")

['prvni', ',', 'druhy', ',', 'treti']

#### Sub a subn
Sub nahrazuje všechny výskyty patternu (1. parametr) v textu (3. parametr) řetězcem z druhého parametru.

In [27]:
re.sub("dog", "cat", "Why does this dog behave as a dog? Bad dog.")

'Why does this cat behave as a cat? Bad cat.'

Pokud pattern v textu není, vrátí se původní text beze změny.

In [29]:
re.sub("hamster", "cat", "Why does this dog behave as a dog? Bad dog.")

'Why does this dog behave as a dog? Bad dog.'

Defaultně se nahradí všechny výskyty patternu. Pokud chceme provést jen omezený počet nahrazení, použijeme parametr count.

In [30]:
re.sub("dog", "cat", "Why does this dog behave as a dog? Bad dog.", count=2)

'Why does this cat behave as a cat? Bad dog.'

Subn se chová podobně jako sub. Pouze vrací nikoli string, ale tuple, v němž je na prvním místě upravený string a na druhém počet provedených náhrad.

In [31]:
re.subn("dog", "cat", "Why does this dog behave as a dog? Bad dog.")

('Why does this cat behave as a cat? Bad cat.', 3)

In [32]:
re.subn("hamster", "cat", "Why does this dog behave as a dog? Bad dog.")

('Why does this dog behave as a dog? Bad dog.', 0)

#### Compile

Zkompiluje pattern do *Pattern* objektu. K čemu je to dobré? Pokud se daný pattern používá ve více regexovských funkcích, musí se pokaždé kompilovat. *Compile* tak ušetří trochu času.  
Situace bez použití *compile*:

In [36]:
without_compile_match = re.finditer("(\\b\\w+)!", original_text)
for match_object in without_compile_match:
    print(match_object.group(1))

while
fair
there
me


Při použití *compile* si musíme pamatovat, že regexové funkce se nyní neberou z balíčku re, ale jsou to metody *Pattern* objektu:

In [38]:
pattern = re.compile("(\\b\\w+)!")
without_compile_match = pattern.finditer(original_text)
for match_object in without_compile_match:
    print(match_object.group(1))

while
fair
there
me


### Příklady použití regexů

#### Vypreparování HTML tagů

Pomiňme otázku, zda jsou pro takovouto úlohu regulární výrazy tím nejvhodnějším přístupem. V patternu máme explicitně zmíněné otevírací a ukončovací zobáčky tagu. Mezi nimi se opakuje grupa. Je v ní OR, přičemž třetí možnost říká: "vezmi jeden znak, který není uvozovka či ukončovací znak tagu". První a druhá možnost ORu se týká případů, kdy je v tagu nějaký řetězec a tak chceme, aby byl ohraničený uvozovkami nejenom na svém začátku, ale i na svém konci. Zde nenastane match jednoho znaku, ale celého slova + uvozovek.

In [8]:
html_tags_text = "blablabla something <b>something importnant</b> blabla <a href='www.some_page.cz'>something</a>   bla"
html_tags_text_matched = re.finditer("<(\"[^\"]*\"|'[^']*'|[^\"'>])*>", html_tags_text)
for match_object in html_tags_text_matched:
    print(match_object.group())

<b>
</b>
<a href='www.some_page.cz'>
</a>


#### Prozkoumání URL
Zde počítáme s tím, že vstupem bude opravdu jen URL, takže používáme stříšku a dolar pro namatchování na začátek a konec textu. U https dáváme za s otazník - vyskytnout se může i http protokol. V hostnamu počítáme s tím, že v něm může být cokoli kromě dvojtečky a lomítka. Je povinný, takže za množinu znaků nedáváme hvězdu, ale plusko. Port včetně uvozující dvojtečky je nepovinný, takže za něj dáváme otazník. Jelikož chceme namatchovat jen jeho číslo a ne onu dvojtečku, máme v grupě další grupu. Ta vnější nás nezajímá a proto na její začátek dáváme "?:". V path už může být cokoli, takže použijeme nejobecnější ".*". 

In [56]:
parsed_url_text = "https://www.some_page.cz:8080/something/file.html"
parsed_url_matched = re.finditer("^https?://([^/:]+)(?::(\\d+))?(/.*)?$", parsed_url_text)
for match_object in parsed_url_matched:
    print("Hostname: ", match_object.group(1))
    if match_object.group(2):
        print("Port: ", match_object.group(2))
    if match_object.group(3):
        print("Path: ", match_object.group(3))

Hostname:  www.some_page.cz
Port:  8080
Path:  /something/file.html


#### Nalezení čísel v textu
Nyní chceme z textu vypreparovat všechna čísla, jak celá, tak desetinná.  
Hned na začátku provádíme OR dělení. Do první škatulky patří "klasická" čísla typu 10 či 2.72. Do druhé se dostanou věci typu .99.  
Věnujme se napřed druhému případu. Mínus je nepovinné, tudíž otazník. Escapujeme desetinnou tečku. Následovat musí aspoň jedna cifra. První případ začíná podobně. Desetinná část tu je nepovinná, proto je v závorkách a za závorkami je otazník. 

In [28]:
numbers_text = "something something 42 bla -80 3.14 bls -1.999999 .15"
numbers_matched = re.finditer("-?[0-9]+(\\.[0-9]+)?|-?\\.[0-9]+", numbers_text)
for match_object in numbers_matched:
    print(match_object.group())


42
-80
3.14
-1.999999
.15


#### Parsování csv

Zde máme díky ORu opět dva mody. V prvním se matchuje řetězec obklopený dvojitými uvozovkami, v druhém zase cokoli, co není čárka nebo dvojitá uvozovka.

In [37]:
csv_text = '10,3.5, ACDC,  "Josef Vyskočil", 789'
csv_matched = re.finditer('\\s*"(?:[^"]|"\\s*")*"\\s*|[^",]+', csv_text)
for match_object in csv_matched:
    print(match_object.group())

10
3.5
 ACDC
  "Josef Vyskočil"
 789


#### Práce se speciálními znaky

Občas se stane, že musíme pracovat s nějakým unicodovým znakem, který na klávesnici nenalezneme. V takovém případě musíme ať už do textu, anebo do patternu napsat escapované u následované kódem příslušného znaku.  
Pokud je kód čtyřznakový, je písmeno *u* malé. Příklad obyčejného vytištění písmene *č*, jehož kód je 010D:

In [17]:
print("\u010D")

č


A jeho použítí v regulárních výrazech:

In [10]:
czech_republic_mathced = re.finditer("\u010Dr", "usa gb sr čr hu")
for match_object in czech_republic_mathced:
    print(match_object.group())

čr


Občas ale člověk narazí na unicodový kód skládající se z pěti znaků. V takovém případě je jednak nutné použít velké U, jednak je potřeba kód zprava doplnit nulami tak, aby měl bez *U* znaků osm. Ukažme si to na příkladu plačícího smajlíka s kódem 1F625.

In [19]:
print("\U0001F625")

😥


A nyní ve větě nahraďme smutného smajlíka smajlíkem veselým.

In [16]:
sad_string = "This string is very very \U0001F625."
print(sad_string)
happy_string = re.sub("\U0001F625", "\U0001F600", sad_string)
print(happy_string)

This string is very very 😥.
This string is very very 😀.


Při přenášení SQL kódu skrze Skype do Teradata SQL asistenta se stává, že si druhý zmíněný software stěžuje na neznámý znak C2 A0. Jedná se o označení pro tzv. [non-breaking space](https://en.wikipedia.org/wiki/Non-breaking_space) zaznamenanou v kódování UTF-8. V unicodu nese tato mezera označení 00A0. Pro nahrazení všech těchto speciálních mezer mezerami normálními použijeme

In [3]:
text_bad_spaces = "This\u00A0is\u00A0some\u00A0strange\u00A0text."
print("Number of bad spaces: " + str(text_bad_spaces.count("\u00A0")))
text_good_spaces = re.sub("\u00A0", " ", text_bad_spaces)
print("Number of bad spaces: " + str(text_good_spaces.count("\u00A0")))

Number of bad spaces: 4
Number of bad spaces: 0


#### Hledání sloupců bez aliasu v SQL

In [7]:
text = ("sloupec1,\n"
        "sloupec2 as druhy,\n"
        "sloupec3 as treti,\n" 
        "sloupec4")
column_with_aliases_matched = re.finditer("^\\w+\\s+as\\s+\\w+,?\\s*$", text, re.MULTILINE)
for match_object in column_with_aliases_matched:
    print(match_object.group())

sloupec2 as druhy,
sloupec3 as treti,


In [29]:
column_without_aliases_matched = re.finditer("^\\w+,?\\s*$", text, re.MULTILINE)
for match_object in column_without_aliases_matched:
    print(match_object.group())

sloupec1,
sloupec4


### Fuzzy search
V případě fuzzy searche hledáme text, který na pattern nemusí sednout na 100%. Jelikož jeho implementace v balíčku *re* chybí, musíme sáhnout po balíčku [regex](https://pypi.org/project/regex/).  
V základu se balíček *regex* snaží chovat podobně jako balíček *re*, tj. najdeme v něm metody *search* anebo třeba *findall*.

In [1]:
import regex

In [9]:
pattern = "pes"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

['pes', 'pes']

Pakliže chceme použít fuzzy search, obalíme pattern do nečíslované grupy. Pokud nám například nevadí nadbytečné inserty, umístíme za grupu složené závorky, do kterých vložíme "i".  
Vidíme, že vrácené bylo například slovo "přes" - když z něj vyhodíme "ř", dostaneme hledané "pes". Další dva výsledky obsahují další písmena nalepená před psem.

In [10]:
pattern = "(?:pes){i}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

['kákal pes', ' přes', 'oves, ten pes']

Pokud bychom chtěli počet povolených nainsertovaných znaků omezit na maximálně 2, změníme zápis ve složených závorkách na "{i<=2}".

In [11]:
pattern = "(?:pes){i<=2}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

['l pes', ' přes', 'n pes']

Pořád ale nedostáváme to, co bychom chtěli. Naštěstí lze s pomocí "(?e)" na začátku patternu vynutit pokus o lepší fit.

In [13]:
pattern = "(?e)(?:pes){i<=2}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

['pes', 'přes', 'pes']

Viděli jsme, že pro inserty existuje přepínač "i". Podobně pro smazání (delete) existuje přepínač "d" a pro substituce přepínač "s". Nakonec přepínač "e" zastupuje všechny fuzzy operace.

In [15]:
pattern = "(?:pes){s<=1}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

['pes', 'řes', 'ves', 'pes', 'pss']

In [16]:
pattern = "(?:pes){d<=1}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

['pes', 'es', 'es', 'pes', 'ps']

In [17]:
pattern = "(?:pes){e<=1}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

[' pes', 'přes', 'ves', ' pes', 'pss']

Lze specifikovat i minimální množství fuzzy operací (ale vždy v kombinaci s maximálním množstvím).

In [20]:
pattern = "(?:pes){1<=s<=3}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

['Ská', 'kal', ' pe', 's, ', 'pře', 's o', 'ves', ', t', 'en ', 'es ', 'pss']

Fuzzy operace lze i kombinovat. V příkladu níže povolujeme maximálně dva inserty, maximálně 2 nahrazení a celkově maximálně tři změny.

In [21]:
pattern = "(?:pes){i<=2,d<=2,e<=3}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

['l pes', ' přes', 'oves', 'te', ' pes', 'pss']

Různým operacím lze dokonce nastavit různou váhu.

In [25]:
pattern = "(?:pes){1s+2i+2d<=2}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

[' pes', 'pře', 'ves', 'ten', 'pes', ' ps']

Nakonec lze i specifikovat, na které znaky v textu (nikoli v patternu) se mohou fuzzy operace aplikovat.

In [28]:
pattern = "(?:pes){s<=2:[a-e]}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

['pes', 'pes']

In [29]:
pattern = "(?:pes){s<=2:[a-es]}"
text = "Skákal pes, přes oves, ten pes pss"
regex.findall(pattern, text)

['pes', 'pes', 'pss']

## Notepad++

K zapnutí regulárních výrazů v Notepadu++ stačí ve Find/replace interfacu zaškrtnout odpovídající checkbox. Pro konstrukci patternů platí víceméně to samé jako v *re*. Hlavní rozdíl spočívá v tom, že zpětná lomítka se v Notepadu++ zdvojovat nemusí. Ostatní odlišnosti jsou spíše drobnějšího rázu. Pár jich tady zmiňme.  
Při hledání okrajů slov se dá použít nejen "\b", které odpovídá začátku i konci slova, ale i "\<" zastupujíící začátek slova a "\>" Zastupující konec slova. Tj. když chceme ve větě
> koneckonců konec ještě neskončil

matchnout slovo "konec", můžeme použít jak pattern
> \bkonec\b

tak pattern
> \\<konec\\>


Lze používat množiny znaků (hranaté závorky) nikoli jen s pomocí zkratek typu "\d", ale s pomocí celých slov. Tudíž například
> \\<[[:alpha:]]*\\>

bude vyhledávat celá slova (a oproti "\w" už nebude matchovat na čísla a podtržítka). Jako další příklady názvů skupin uveďme *alnum*, *blank* či *digit*.

Pokud se dá vyhledat znak podle unicodového kódu, tak jsem nepřišel na to jak. Vyhledávat se ale dá pomocí hexadecimálního kódu zapsaného do složených závorek, před které se umístí "\x". Například když chceme nalézt písmeno č, použijeme jeho hexadecimální zápis v UTF-16 (a to i když máme nastavené kódování UTF-8), kterým je 0x010D. Ve výsledku má pattern podobu  
> \x{010D}

Též bychom mohli u search modu přejít z regulárních výrazů na "extended" a hledat "č" pomocí \u010D.  

Podobně když budeme chtít non-breaking mezeru nahradit mezerou normální, budeme (při zaškrtnutém "Regular expression") nahrazovat znak \x{00A0}. Samozřejmě v tomto případě bude lepší nahrazovat "\s+" - kód si asi nikdo pamatovat nebude. Mimochodem zde nesmíme použít místo plusu hvězdičku - takovýto pattern by byl platný všude a tak by byl **každý** znak textu následován mezerou.  
Nicméně takovýto postup nahradí i znaky nových řádků či tabulátory, které text činí přehlednějším. Proto asi nejrozumnější bude použít pattern \[^\S\n\t\]+. Ten se chytne na všechny znaky mimo těch, které jsou "nebíle", jsou znak nového řádku anebo jsou tabulátor.

## Teradata
Z hlediska znaků v patternu Teradata podporuje stejné věci jako Python - tj. vše od * a ? přes "\w" a "\b" až po zpětné reference na grupu funguje. Podívejme se tedy na funkce, které s patterny pracují.  
Zdůrazněme, že tyto funkce má smysl použít v selectech, avšak ve wheru nebo v joinovacích podmínkách by rychlost zpracování byla bídná.
### REGEXP_SUBSTR
Funkce *REGEXP_SUBSTR* je ekvivalentem pythoního *re.search*. Tj. najde první výskyt patternu (2. argument) v textu (1. argument). 
Například
```SQL
SELECT REGEXP_SUBSTR('Do you like hamsters?', '(dog|cat|hamster)');
```
vrací *hamster*.  
Příklad použití v normálním dotazu, kdy chceme z řetězce ve sloupci patent_name vzít až anglickou sekci uvozenou pomocí "EN:":
```SQL
select
   patent_submission_year,
   patent_name,
   REGEXP_SUBSTR(patent_name, 'EN:.*')
from wsp_ai.vsa_patents
```

Volitelnými parametry funkce jsou 
- pozice v textu, od které hledání začíná; defaultní je 1, což u Teradaty znamená první znak (narozdíl od Pythonu se nezačíná od indexu 0).
- kolikátý výskyt patternu se má vrátit. Opravdu se vrací jen n-tý výskyt, nikoli n výskytů! Pokud je n větší než faktický počet výskytů patternu, vrátí se NULL.   
   Příklad:  
   ```SQL
   SELECT REGEXP_SUBSTR('Do you like hamsters or cats?', '(dog|cat|hamster)',1,2);
   ```
   vrací *cat*.
   ```SQL
   SELECT REGEXP_SUBSTR('Do you like hamsters or cats?', '(dog|cat|hamster)',1,3);
   ```
   vrací *NULL*
- flagy:
    - 'c': rozlišování velikosti písmen (defaultní)  
    - 'i': nerozlišování velikosti písmen
    - 'n': tečka může matchnout i znak odřádkování (defaultně to nejde)
    - 'm': text je braný jako multiřádkový, tj. stříška a dolar se matchne nejen na začátek a konec textu (což je default), ale i na začátky a konce řádků
    - 'x': ignorování bílých znaků v patternu. tj.
    ```SQL
    SELECT REGEXP_SUBSTR('Hamster parrot catdog', 'cat dog',1,1, 'x');
    ```
    vrátí catdog, avšak
    ```SQL
    SELECT REGEXP_SUBSTR('Hamster parrot cat dog', 'catdog',1,1, 'x');
    ```
    vrátí NULL. Hypoteticky by to mohlo být užitečné pro zpřehlednění patternu.  
   
    Obecně můžeme mít flagů najednou více, například
    ```SQL
    SELECT REGEXP_SUBSTR('Hamster parrot CATdog', 'cat dog',1,1, 'xi');
    ```

Jinak když chceme použít n+1-tý parametr, musíme použít i n-tý parametr. Tj. když chceme použít flagy, musíme uvést i pozici v textu a to, kolikátý výskyt patternu v textu požadujeme.

### REGEX_REPLACE
Jedná se o obdobu pythoního re.sub, tj. nahrazuje výskyty patternu (2. parametr) v textu (1. parametr) řetězcem v 3. parametru.  
Příklad:
```SQL
SELECT REGEXP_REPLACE('Do you like hamsters?', 'h[a-z]+s?', 'cats');
```
vrátí "Do you like cats?"  
Stejně jak v předchozím případě se i zde vyskytují tři nepovinné parametry - index, od kterého začne funkce text zkoumat, pořadí výskytu patternu, který je nahrazen, a flagy. Jako příklad si vezměme situaci bez dodatečných parametrů:
```SQL
SELECT REGEXP_REPLACE('Hamsters... hamsters... hamsters are everywhere', 'hamsters', 'HAMSTERS');
```
Vrácena bude věta "Hamsters... HAMSTERS... HAMSTERS are everywhere". Tj. nahrazování je case sensitive a nahrazeny jsou všechny výskyty patternu. Nyní řekneme, že chceme case insensitive chování a chceme nahradit jen druhý výskyt patternu:
```SQL
SELECT REGEXP_REPLACE('Hamsters... hamsters... hamsters are everywhere', 'hamsters', 'HAMSTERS', 1, 2, 'i');
```
Výsledkem bude "Hamsters... HAMSTERS... hamsters are everywhere".


### REGEX_INSTR
Zde máme co do činění s bratrancem *REGEXP_SUBSTR*. Zatímco ale ten vracel řetězec v textu, který odpovídal patternu. *REGEX_INSTR* vrací index, na kterém tento řetězec v textu začíná.  Tj. například
```SQL
SELECT REGEXP_INSTR('Hamsters... hamsters... hamsters are everywhere', 'hamsters');
```
vrátí třináctku.  
Volitelné parametry jsou tentokrát čtyři. První odpovídá počáteční pozici hledání, druhý počtu hledaných výskytů patternu a čtvrtý flagům. Novinkou je parametr třetí, který může nabýt hodnot 0 či 1. Defaultní nula znamená, že uživatel chce vrátit index, kde výskyt patternu v textu začíná; jednička značí, že uživatel chce znát index, kde výskyt patternu končí.

### REGEX_SIMILAR
Tato funkce porovnává text (1. parametr) a pattern (2. parametr). Pokud jsou stejné, vrátí jedničku, v opačném případě uživatel obdrží nulu. Vyskytuje se zde jenom jeden volitelný parametr - flagy.
Příklad:
```SQL
SELECT REGEXP_SIMILAR('Hamsters', 'hamsters');
```
vrátí nulu,
```SQL
SELECT REGEXP_SIMILAR('Hamsters', '(h|H)amsters');
```
vrátí jedničku.

### REGEX_SPLIT_TO_TABLE
Funkce umožňuje vzít řetězec a na základě přitomnosti patternu ho rozdělit na několik kusů. Tyto kusy posléze budou bydlet na samostatných řádcích vytvořené tabulky.
Tj. toto
```SQL
SELECT * from table 
  ( 
      regexp_split_to_table('animals', 'dog,cat;hamster|horse','[,;|]','c') 
	  returns (identifier varchar(100), row_no integer, entity varchar(100))
  ) as t1;
```
vede na  

|identifier|row_no|entity|
|-|-|-|
|animals|1|dog|
|animals|2|cat|
|animals|3|hamster|
|animals|4|horse|

Řetězce, pattterny i flagy lze brát všechny z tabulek v databázi. Dejme tomu, že v kontejneru wsp_ai máme tabulku split_table_example o následujícím obsahu:

|id_text|silly_text|parsing_pattern|regex_flag|
|-|-|-|-|
|languages|python12java13c++14scala|[0-9]+|c|
|animals|dog34cat45hamster76horse|[0-9]+|c|

Když do teradatího SQL asistenta napíšeme 
```SQL
SELECT * from table 
  ( 
      regexp_split_to_table(wsp_ai.split_table_example.id_text,
                            wsp_ai.split_table_example.silly_text, 
                            wsp_ai.split_table_example.parsing_pattern,
                            wsp_ai.split_table_example.regex_flag) 
	  returns (identifier varchar(100), row_no integer, entity varchar(100))
  ) as t1
order by t1.identifier, t1.row_no;
```
obdržíme

|identifiers|row_no|entity|
|-|-|-|
|animals |1|dog|
|animals |2|cat|
|animals |3|hamster|
|animals |4|horse|
|languages|1|python|
|languages|2|java|
|languages|3|c++|
|languages|4|scala|
